In [40]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null

import shutil,os
import subprocess
import random,torch
import whisper

torch.manual_seed(42)
random.seed(42)

In [41]:
"""
# define model parameters and load model
# Transformer based Seq2Seq model trained on weak supervision
"""
model_size = "medium"     # ['tiny', 'base', 'small', 'medium', 'large']
language = "English"      # ['any', 'English']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'

model = whisper.load_model(model_size)

In [ ]:
# upload audio file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [53]:
def generate_transcript(path):

  # copy file to current working directory from google drive

  shutil.copy(path,os.getcwd())
  path = path.split("/")[-1]
  filename = path[:-4].split(' ')
  if len(filename)==1:
    filename = filename[0]
  else:
    filename=f"{filename[0]}{filename[1]}"
  
  # if file is not in wav format convert it to wave format
  if path[-3:] != 'wav':
    subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
    os.remove(path)
    path = 'audio.wav'

  print("File conversion - Done,Now Transcribing")

  """
    Performs speaker verification using cosine distance between speaker embeddings
    
    # dict_keys(['text', 'segments', 'language'])
    id : id of audio chunk/segment 
    start & end : start & end time of audio chunk
    text : transcribed text
    tokens : generated token list
    temperature : non-deterministic factor (0 means deterministic model)
    avg_logprob : uses additive margin softmax loss 
    compression_ratio : .WAV is windows compressed but .wav is uncompressed
    no_speech_prob : probability of silence detection
  """

  result = model.transcribe(path)
  segments = result["segments"]


  f = open(f"{filename}.txt", "w")

  text = ""
  """
  segment : new addition words : list and speaker : str
  """
  for (i, segment) in enumerate(segments):
      try:
        f.write(segment["text"][1:] + '\n\n') 
      except:
        continue
  f.close()

  # remove audio.wav
  os.remove("audio.wav")

  print(f"Transcription - Done,check {filename}.txt")


change to filename you want to transcribe - "/content/drive/MyDrive/Audios/April 2.WAV"

In [54]:

path = "/content/drive/MyDrive/Audios/April 2.WAV"

generate_transcript(path)

File conversion - Done,Now Transcribing
Transcription - Done,check April2.txt


In [39]:
#